In [1]:
import numpy as np
import logging
from gen_inst import TSPInstance, load_dataset
from gls import guided_local_search
import time
import elkai
from tqdm import tqdm

In [2]:
perturbation_moves_map = {
    20: 5,
    50: 30,
    100: 40,
    200: 40,
}
iter_limit_map = {
    20: 73,
    50: 175,
    100: 1800,
    200: 800,
}
SCALE = 1000000
test_sizes = list(iter_limit_map.keys())

In [3]:
def heuristics_reevo(distance_matrix: np.ndarray) -> np.ndarray:
    # Calculate the average distance for each node
    average_distance = np.mean(distance_matrix, axis=1)

    # Calculate the distance ranking for each node
    distance_ranking = np.argsort(distance_matrix, axis=1)
    
    # Calculate the mean of the closest distances for each node
    closest_mean_distance = np.mean(distance_matrix[np.arange(distance_matrix.shape[0])[:, None], distance_ranking[:, 1:5]], axis=1)

    # Initialize the indicator matrix and calculate ratio of distance to average distance
    indicators = distance_matrix / average_distance[:, np.newaxis]

    # Set diagonal elements to np.inf
    np.fill_diagonal(indicators, np.inf)

    # Adjust the indicator matrix using the statistical measure
    indicators += closest_mean_distance[:, np.newaxis] / np.sum(distance_matrix, axis=1)[:, np.newaxis]

    return indicators

def vanilla_ktsp(distance_matrix: np.ndarray) -> np.ndarray:
    return distance_matrix
from sklearn.neighbors import KernelDensity



def heuristics(distance_matrix):
    smallest_distance = np.min(distance_matrix[distance_matrix != 0])
    standard_deviation = np.std(distance_matrix[distance_matrix != 0])
    
    # Calculate scores for each edge based on how much they exceed the smallest distance
    # and adjust scores by standard deviation, so smaller distances have smaller scores
    scores = (distance_matrix - smallest_distance) / (standard_deviation + 1e-5) 
    
    return scores


In [4]:
def calculate_cost(inst: TSPInstance, path: np.ndarray):
    return inst.distmat[path, np.roll(path, 1)].sum().item()

In [5]:
# Precompute the optimal solutions
optimal_objs_dict = {}
for problem_size in test_sizes:
    dataset_path = f"dataset/test{problem_size}_dataset.npy"
    dataset = load_dataset(dataset_path)
    n_instances = dataset[0].n
    logging.info(f"[*] Evaluating {dataset_path} with LKH")

    optimal_objs = []
    for i, instance in enumerate(tqdm(dataset, desc=f"tsp{problem_size}")):
        elkai_dist = elkai.DistanceMatrix(((instance.distmat * SCALE).astype(int)).tolist()) 
        optimal_route = elkai_dist.solve_tsp() # e.g. [0, 2, 1, 0]; with proven optimal solutions up to N=315 (https://github.com/fikisipi/elkai)
        optimal_obj = calculate_cost(instance, np.array(optimal_route[:-1]))
        optimal_objs.append(optimal_obj)
        
    mean_optimal_obj = np.mean(optimal_objs)

    optimal_objs_dict[problem_size] = mean_optimal_obj
    
print(optimal_objs_dict)

tsp200: 100%|██████████| 64/64 [01:23<00:00,  1.31s/it]

{20: 3.8039483761359274, 50: 5.754330442860288, 100: 7.726043475790581, 200: 10.695986353812152}


In [6]:
optimal_objs_dict = {20: 3.811766231009382, 50: 5.604237036311405, 100: 7.747348608599886, 200: 10.715539641334345}

def solve(inst: TSPInstance, heuristics):
    start_time = time.time()
    heu = heuristics(inst.distmat.copy())
    result = guided_local_search(inst.distmat, heu, perturbation_moves_map[inst.n], iter_limit_map[inst.n])
    duration = time.time() - start_time
    return calculate_cost(inst, result), duration

def evaluate(function):
    print("[*] Function:", function.__name__, "\n")
    for problem_size in iter_limit_map.keys():
        dataset_path = f"dataset/test{problem_size}_dataset.npy"
        dataset = load_dataset(dataset_path)
        logging.info(f"[*] Evaluating {dataset_path}")

        objs = []
        durations = []
        for instance in tqdm(dataset):
            obj, duration = solve(instance, function)
            objs.append(obj)
            durations.append(duration)

        mean_obj = np.mean(objs).item()
        mean_optimal_obj = optimal_objs_dict[problem_size]
        gap = mean_obj / mean_optimal_obj - 1
        print(f"[*] Average for {problem_size}: {mean_obj:.6f} ({mean_optimal_obj:.6f})")
        print(f"[*] Optimality gap: {gap*100:.6f}%")
        print(f"[*] Total/Average duration: {sum(durations):.6f}s {sum(durations)/len(durations):.6f}s")
        print()

In [ ]:
evaluate(heuristics_reevo)

[*] Function: heuristics_reevo 


100%|██████████| 64/64 [00:00<00:00, 1425.17it/s]


[*] Average for 20: 3.811971 (3.811766)
[*] Optimality gap: 0.005366%
[*] Total/Average duration: 0.042877s 0.000670s


100%|██████████| 64/64 [00:01<00:00, 46.98it/s]


[*] Average for 50: 5.604389 (5.604237)
[*] Optimality gap: 0.002716%
[*] Total/Average duration: 1.349973s 0.021093s


 70%|███████   | 45/64 [00:43<00:19,  1.01s/it]

In [8]:
evaluate(vanilla_ktsp)

[*] Function: vanilla_ktsp 


100%|██████████| 64/64 [00:00<00:00, 1541.49it/s]


[*] Average for 20: 3.811972 (3.836285)
[*] Optimality gap: -0.633778%
[*] Total/Average duration: 0.041518s 0.000649s


100%|██████████| 64/64 [00:01<00:00, 47.93it/s]


[*] Average for 50: 5.604631 (5.684580)
[*] Optimality gap: -1.406423%
[*] Total/Average duration: 1.322401s 0.020663s


100%|██████████| 64/64 [01:01<00:00,  1.04it/s]


[*] Average for 100: 7.747342 (7.778580)
[*] Optimality gap: -0.401598%
[*] Total/Average duration: 61.345289s 0.958520s


100%|██████████| 64/64 [01:42<00:00,  1.60s/it]

[*] Average for 200: 10.740516 (10.711946)
[*] Optimality gap: 0.266710%
[*] Total/Average duration: 102.087550s 1.595118s


In [12]:
evaluate(heuristics)

[*] Function: heuristics 


100%|██████████| 64/64 [00:00<00:00, 801.04it/s]


[*] Average for 20: 3.812049 (3.811766)
[*] Optimality gap: 0.007423%
[*] Total/Average duration: 0.078726s 0.001230s


100%|██████████| 64/64 [00:01<00:00, 39.93it/s]


[*] Average for 50: 5.604527 (5.604237)
[*] Optimality gap: 0.005181%
[*] Total/Average duration: 1.591597s 0.024869s


100%|██████████| 64/64 [01:51<00:00,  1.73s/it]


[*] Average for 100: 7.747608 (7.747349)
[*] Optimality gap: 0.003352%
[*] Total/Average duration: 110.894471s 1.732726s


100%|██████████| 64/64 [04:31<00:00,  4.24s/it]

[*] Average for 200: 10.739542 (10.715540)
[*] Optimality gap: 0.223996%
[*] Total/Average duration: 271.578539s 4.243415s


[*] Function: heuristic 


100%|██████████| 64/64 [00:00<00:00, 675.94it/s]


[*] Average for 20: 3.811972 (3.811766)
[*] Optimality gap: 0.005394%
[*] Total/Average duration: 0.094683s 0.001479s


100%|██████████| 64/64 [00:01<00:00, 39.25it/s]


[*] Average for 50: 5.604631 (5.604237)
[*] Optimality gap: 0.007024%
[*] Total/Average duration: 1.619518s 0.025305s


100%|██████████| 64/64 [01:03<00:00,  1.01it/s]


[*] Average for 100: 7.747342 (7.747349)
[*] Optimality gap: -0.000089%
[*] Total/Average duration: 63.327086s 0.989486s


100%|██████████| 64/64 [02:06<00:00,  1.98s/it]

[*] Average for 200: 10.740516 (10.715540)
[*] Optimality gap: 0.233084%
[*] Total/Average duration: 126.387626s 1.974807s


[*] Function: heuristics1 


100%|██████████| 64/64 [00:00<00:00, 1482.38it/s]


[*] Average for 20: 3.811972 (3.811766)
[*] Optimality gap: 0.005394%
[*] Total/Average duration: 0.042144s 0.000659s


100%|██████████| 64/64 [00:01<00:00, 48.33it/s]


[*] Average for 50: 5.604631 (5.604237)
[*] Optimality gap: 0.007024%
[*] Total/Average duration: 1.308296s 0.020442s


100%|██████████| 64/64 [01:00<00:00,  1.05it/s]


[*] Average for 100: 7.747378 (7.747349)
[*] Optimality gap: 0.000382%
[*] Total/Average duration: 60.789418s 0.949835s


100%|██████████| 64/64 [01:41<00:00,  1.58s/it]

[*] Average for 200: 10.739849 (10.715540)
[*] Optimality gap: 0.226862%
[*] Total/Average duration: 101.080393s 1.579381s


[*] Function: heuristics2 


100%|██████████| 64/64 [00:00<00:00, 648.87it/s]


[*] Average for 20: 3.811972 (3.811766)
[*] Optimality gap: 0.005394%
[*] Total/Average duration: 0.096629s 0.001510s


100%|██████████| 64/64 [00:01<00:00, 42.33it/s]


[*] Average for 50: 5.604389 (5.604237)
[*] Optimality gap: 0.002716%
[*] Total/Average duration: 1.497122s 0.023393s


100%|██████████| 64/64 [01:03<00:00,  1.01it/s]


[*] Average for 100: 7.747413 (7.747349)
[*] Optimality gap: 0.000834%
[*] Total/Average duration: 63.294463s 0.988976s


100%|██████████| 64/64 [01:42<00:00,  1.61s/it]

[*] Average for 200: 10.745300 (10.715540)
[*] Optimality gap: 0.277730%
[*] Total/Average duration: 102.730486s 1.605164s


[*] Function: heuristics3 


100%|██████████| 64/64 [00:00<00:00, 866.90it/s]


[*] Average for 20: 3.811766 (3.811766)
[*] Optimality gap: -0.000000%
[*] Total/Average duration: 0.070565s 0.001103s


100%|██████████| 64/64 [00:01<00:00, 43.81it/s]


[*] Average for 50: 5.604389 (5.604237)
[*] Optimality gap: 0.002716%
[*] Total/Average duration: 1.450111s 0.022658s


100%|██████████| 64/64 [01:02<00:00,  1.02it/s]


[*] Average for 100: 7.747342 (7.747349)
[*] Optimality gap: -0.000089%
[*] Total/Average duration: 62.863948s 0.982249s


100%|██████████| 64/64 [01:46<00:00,  1.66s/it]

[*] Average for 200: 10.742387 (10.715540)
[*] Optimality gap: 0.250545%
[*] Total/Average duration: 106.140208s 1.658441s


[*] Function: heuristics4 


100%|██████████| 64/64 [00:00<00:00, 1459.42it/s]


[*] Average for 20: 3.811972 (3.811766)
[*] Optimality gap: 0.005394%
[*] Total/Average duration: 0.043853s 0.000685s


100%|██████████| 64/64 [00:01<00:00, 47.23it/s]


[*] Average for 50: 5.604631 (5.604237)
[*] Optimality gap: 0.007024%
[*] Total/Average duration: 1.338711s 0.020917s


100%|██████████| 64/64 [01:01<00:00,  1.04it/s]


[*] Average for 100: 7.747378 (7.747349)
[*] Optimality gap: 0.000382%
[*] Total/Average duration: 61.681760s 0.963778s


100%|██████████| 64/64 [01:42<00:00,  1.61s/it]

[*] Average for 200: 10.739849 (10.715540)
[*] Optimality gap: 0.226862%
[*] Total/Average duration: 102.922113s 1.608158s
